In [1]:
import pandas as pd
# !pip install gensim
import numpy as np
import gensim
import os
import string
import re
import pprint
from gensim import corpora, models, similarities
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.manifold import TSNE
import nltk
from nltk import word_tokenize, tokenize
nltk.download('punkt')

%matplotlib inline
from matplotlib import pyplot as plt
#!pip install vaderSentiment
import datetime
from collections import Counter
from tqdm import tqdm
tqdm.pandas(desc='progress-bar')
from gensim.models.doc2vec import LabeledSentence
import operator
from nltk.stem.porter import *
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\CK\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
C:\Users\CK\Anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


### Getting dataframes from cleaned csvs

In [2]:
trump_df = pd.read_csv('../data/clean_trump.csv').drop(columns = ['Unnamed: 0'])
print(trump_df.shape)
trump_df.head()

(125783, 6)


,user_id,tweet,state,likes,retweet_count,avg_polarity
0,3.606665e+08,#Elecciones2020 | En #Florida: #JoeBiden dice ...,Florida,0.0,0.0,0.000000
1,8.436472e+06,"#Trump: As a student I used to hear for years,...",Oregon,2.0,1.0,0.461917
2,4.741380e+07,You get a tie! And you get a tie! #Trump ‘s ra...,District of Columbia,4.0,3.0,0.000000
3,1.138416e+09,@CLady62 Her 15 minutes were over long time ag...,California,2.0,0.0,-0.323204
4,9.007611e+17,@DeeviousDenise @realDonaldTrump @nypost There...,Ohio,0.0,0.0,-0.041564


In [3]:
biden_df = pd.read_csv('../data/clean_biden.csv').drop(columns = ['Unnamed: 0'])
print(biden_df.shape)
biden_df.head()

(96528, 6)


,user_id,tweet,state,likes,retweet_count,avg_polarity
0,3.606665e+08,#Elecciones2020 | En #Florida: #JoeBiden dice ...,Florida,0.0,0.0,0.000000
1,3.494182e+09,@IslandGirlPRV @BradBeauregardJ @MeidasTouch T...,Montana,0.0,0.0,0.000000
2,1.032807e+18,#censorship #HunterBiden #Biden #BidenEmails #...,California,1.0,0.0,0.000000
3,1.994033e+07,"In 2020, #NYPost is being #censorship #CENSORE...",Illinois,0.0,0.0,-0.339855
4,9.607387e+17,"FBI Allegedly Obtained Hunter Biden Computer, ...",Kentucky,0.0,0.0,-0.050000


In [4]:
trump_df['hashtag'] = 'trump'
biden_df['hashtag'] = 'biden'

full_df = pd.concat([trump_df, biden_df], ignore_index=True)
full_df = full_df.drop_duplicates(subset='tweet', keep=False)
display(full_df.head())

trump_df=full_df[full_df['hashtag']=='trump']
biden_df=full_df[full_df['hashtag']=='biden']
print(trump_df.shape)
print(biden_df.shape)
display(trump_df.head())
display(biden_df.head())
len(set(full_df['tweet']))
og_tweet=full_df['tweet']

,user_id,tweet,state,likes,retweet_count,avg_polarity,hashtag
1,8.436472e+06,"#Trump: As a student I used to hear for years,...",Oregon,2.0,1.0,0.461917,trump
2,4.741380e+07,You get a tie! And you get a tie! #Trump ‘s ra...,District of Columbia,4.0,3.0,0.000000,trump
3,1.138416e+09,@CLady62 Her 15 minutes were over long time ag...,California,2.0,0.0,-0.323204,trump
4,9.007611e+17,@DeeviousDenise @realDonaldTrump @nypost There...,Ohio,0.0,0.0,-0.041564,trump
5,5.404769e+08,One of the single most effective remedies to e...,Pennsylvania,0.0,0.0,0.366371,trump


(104196, 7)
(76653, 7)


,user_id,tweet,state,likes,retweet_count,avg_polarity,hashtag
1,8.436472e+06,"#Trump: As a student I used to hear for years,...",Oregon,2.0,1.0,0.461917,trump
2,4.741380e+07,You get a tie! And you get a tie! #Trump ‘s ra...,District of Columbia,4.0,3.0,0.000000,trump
3,1.138416e+09,@CLady62 Her 15 minutes were over long time ag...,California,2.0,0.0,-0.323204,trump
4,9.007611e+17,@DeeviousDenise @realDonaldTrump @nypost There...,Ohio,0.0,0.0,-0.041564,trump
5,5.404769e+08,One of the single most effective remedies to e...,Pennsylvania,0.0,0.0,0.366371,trump


,user_id,tweet,state,likes,retweet_count,avg_polarity,hashtag
125784,3.494182e+09,@IslandGirlPRV @BradBeauregardJ @MeidasTouch T...,Montana,0.0,0.0,0.00000,biden
125785,1.032807e+18,#censorship #HunterBiden #Biden #BidenEmails #...,California,1.0,0.0,0.00000,biden
125787,9.607387e+17,"FBI Allegedly Obtained Hunter Biden Computer, ...",Kentucky,0.0,0.0,-0.05000,biden
125789,1.165118e+18,In an effort to find the truth about allegatio...,Illinois,1.0,0.0,0.27115,biden
125790,1.139962e+18,Twitter is doing everything they can to help D...,California,1.0,0.0,0.12135,biden


### Further tweet cleaning - get rid of @user and simplify words for word2vec, doc2vec using Porter Stemmer

In [5]:
def remove_mentions(df):
    
    pattern = r"@[\w]*"
    tweets = df['tweet']
    
    clean_tweets = []
    for tweet in tweets:
        r = re.findall(pattern, tweet)
        for i in r:
            tweet = re.sub(i, '', tweet)
        clean_tweets.append(tweet)
            
    return clean_tweets

In [6]:
trump_df['og_tweet'] = trump_df['tweet']

trump_df['tweet'] = remove_mentions(trump_df)
trump_df['tweet'] = trump_df['tweet'].str.replace("[^a-zA-Z#]", " ")
trump_df['tweet'] = trump_df['tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3]))
trump_tokens = trump_df['tweet'].apply(lambda x: x.split())
trump_tokens[:5]
trump_df.head()

C:\Users\CK\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\CK\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\CK\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

,user_id,tweet,state,likes,retweet_count,avg_polarity,hashtag,og_tweet
1,8.436472e+06,#Trump student used hear years years heard Chi...,Oregon,2.0,1.0,0.461917,trump,"#Trump: As a student I used to hear for years,..."
2,4.741380e+07,#Trump rally #Iowa https jJalUUmh,District of Columbia,4.0,3.0,0.000000,trump,You get a tie! And you get a tie! #Trump ‘s ra...
3,1.138416e+09,minutes were over long time Omarosa never repr...,California,2.0,0.0,-0.323204,trump,@CLady62 Her 15 minutes were over long time ag...
4,9.007611e+17,There many them Unless have been voting more t...,Ohio,0.0,0.0,-0.041564,trump,@DeeviousDenise @realDonaldTrump @nypost There...
5,5.404769e+08,single most effective remedies eradicate anoth...,Pennsylvania,0.0,0.0,0.366371,trump,One of the single most effective remedies to e...


In [7]:
biden_df['og_tweet']=biden_df['tweet']
biden_df['tweet'] = remove_mentions(biden_df)
biden_df['tweet'] = biden_df['tweet'].str.replace("[^a-zA-Z#]", " ")
biden_df['tweet'] = biden_df['tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3]))
biden_tokens = biden_df['tweet'].apply(lambda x: x.split())
biden_tokens[:5]

C:\Users\CK\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\CK\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\CK\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

125784    [This, #Biden, made, #TrumpIsNotAmerica, https...
125785    [#censorship, #HunterBiden, #Biden, #BidenEmai...
125787    [Allegedly, Obtained, Hunter, Biden, Computer,...
125789    [effort, find, truth, about, allegations, that...
125790    [Twitter, doing, everything, they, help, Democ...
Name: tweet, dtype: object

In [8]:
stemmer = PorterStemmer()

In [9]:
trump_tokens = trump_tokens.apply(lambda x: [stemmer.stem(i) for i in x])
trump_tokens[:5]

1    [#trump, student, use, hear, year, year, heard...
2               [#trump, ralli, #iowa, http, jjaluumh]
3    [minut, were, over, long, time, omarosa, never...
4    [there, mani, them, unless, have, been, vote, ...
5    [singl, most, effect, remedi, erad, anoth, rou...
Name: tweet, dtype: object

In [10]:
biden_tokens = biden_tokens.apply(lambda x: [stemmer.stem(i) for i in x])
biden_tokens[:5]

125784    [thi, #biden, made, #trumpisnotamerica, http, ...
125785    [#censorship, #hunterbiden, #biden, #bidenemai...
125787    [allegedli, obtain, hunter, biden, comput, dat...
125789    [effort, find, truth, about, alleg, that, allo...
125790    [twitter, do, everyth, they, help, democrat, e...
Name: tweet, dtype: object

In [11]:
trump_tokens_list = trump_tokens.tolist()
for i in range(len(trump_tokens)):
    trump_tokens_list[i] = ' '.join(trump_tokens_list[i])
trump_df['tweet'] = trump_tokens_list
trump_df.head()

C:\Users\CK\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,user_id,tweet,state,likes,retweet_count,avg_polarity,hashtag,og_tweet
1,8.436472e+06,#trump student use hear year year heard china ...,Oregon,2.0,1.0,0.461917,trump,"#Trump: As a student I used to hear for years,..."
2,4.741380e+07,#trump ralli #iowa http jjaluumh,District of Columbia,4.0,3.0,0.000000,trump,You get a tie! And you get a tie! #Trump ‘s ra...
3,1.138416e+09,minut were over long time omarosa never repres...,California,2.0,0.0,-0.323204,trump,@CLady62 Her 15 minutes were over long time ag...
4,9.007611e+17,there mani them unless have been vote more tha...,Ohio,0.0,0.0,-0.041564,trump,@DeeviousDenise @realDonaldTrump @nypost There...
5,5.404769e+08,singl most effect remedi erad anoth round #tru...,Pennsylvania,0.0,0.0,0.366371,trump,One of the single most effective remedies to e...


In [12]:
biden_tokens_list=biden_tokens.tolist()

for i in range(len(biden_tokens)):
    biden_tokens_list[i] = ' '.join(biden_tokens_list[i])
biden_df['tweet']  = biden_tokens_list
biden_df.head()

C:\Users\CK\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,user_id,tweet,state,likes,retweet_count,avg_polarity,hashtag,og_tweet
125784,3.494182e+09,thi #biden made #trumpisnotamerica http ubqafu,Montana,0.0,0.0,0.00000,biden,@IslandGirlPRV @BradBeauregardJ @MeidasTouch T...
125785,1.032807e+18,#censorship #hunterbiden #biden #bidenemail #b...,California,1.0,0.0,0.00000,biden,#censorship #HunterBiden #Biden #BidenEmails #...
125787,9.607387e+17,allegedli obtain hunter biden comput data ukra...,Kentucky,0.0,0.0,-0.05000,biden,"FBI Allegedly Obtained Hunter Biden Computer, ..."
125789,1.165118e+18,effort find truth about alleg that allow peopl...,Illinois,1.0,0.0,0.27115,biden,In an effort to find the truth about allegatio...
125790,1.139962e+18,twitter do everyth they help democrat elect ma...,California,1.0,0.0,0.12135,biden,Twitter is doing everything they can to help D...


### Splitting data into training and testing sets, applying labels based on `avg_polarity`

##### Using threshold of 0.05 as 'undecided' from previous VADER and textblod analyses. Maintaining absolute netural polarity with those who scored near 0.0 for polarity.

In [13]:
threshold = 0.05
train_trump_df = trump_df[(trump_df['avg_polarity'] < -threshold) | (trump_df['avg_polarity'] > threshold) | (trump_df['avg_polarity'] == 0)]
test_trump_df = trump_df[((trump_df['avg_polarity'] >= -threshold) & (trump_df['avg_polarity'] <= threshold)) & (trump_df['avg_polarity'] != 0)]
print("Training size: ", train_trump_df.shape)
print("Training negatives: ", train_trump_df[train_trump_df['avg_polarity']<-threshold].shape)
print("Training positives: ", train_trump_df[train_trump_df['avg_polarity']>threshold].shape)
print("Training neutrals: ", train_trump_df[train_trump_df['avg_polarity']==0.0].shape)

print("Testing size: ", test_trump_df.shape)



Training size:  (98498, 8)
Training negatives:  (38433, 8)
Training positives:  (38975, 8)
Training neutrals:  (21090, 8)
Testing size:  (5698, 8)


In [14]:
labels=[]
for index, row in train_trump_df.iterrows():
    if row['avg_polarity'] == 0:
        labels.append(0)
    elif row['avg_polarity'] < 0:
        labels.append(1)
    else:
        labels.append(-1)

train_trump_df['label'] = labels
test_trump_df.head()

C:\Users\CK\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,user_id,tweet,state,likes,retweet_count,avg_polarity,hashtag,og_tweet
4,9.007611e+17,there mani them unless have been vote more tha...,Ohio,0.0,0.0,-0.041564,trump,@DeeviousDenise @realDonaldTrump @nypost There...
29,8.436472e+06,#trump mind script right thi stuff want hear a...,Oregon,3.0,1.0,-0.003093,trump,"#Trump: You don't mind if I go off script, rig..."
33,1.087710e+09,well #kayleighmcenani taken twitter thi post r...,Georgia,3.0,4.0,-0.012900,trump,Well #kayleighmcenany was taken off of Twitter...
103,1.743698e+08,singl from demi lovato #commanderinchief caus ...,California,1.0,1.0,0.016378,trump,"The new single from Demi Lovato, #CommanderInC..."
147,8.436472e+06,thi time next week #trump go insist that didn ...,Oregon,1.0,1.0,0.012500,trump,By this time next week #Trump is going to be i...


In [15]:
train_biden_df = biden_df[(biden_df['avg_polarity'] < -threshold) | (biden_df['avg_polarity'] > threshold) | (biden_df['avg_polarity'] == 0.0)]
test_biden_df = biden_df[((biden_df['avg_polarity'] >= -threshold) & (biden_df['avg_polarity'] <= threshold)) & (biden_df['avg_polarity'] != 0)]

print("Training size: ", train_biden_df.shape)
print("Training negatives: ", train_biden_df[train_biden_df['avg_polarity']<-threshold].shape)
print("Training positives: ", train_biden_df[train_biden_df['avg_polarity']>threshold].shape)
print("Training neutrals: ", train_biden_df[train_biden_df['avg_polarity']==0.0].shape)
print("Testing size: ", test_biden_df.shape)
test_biden_df.head()
biden_df.shape

Training size:  (72765, 8)
Training negatives:  (21296, 8)
Training positives:  (30920, 8)
Training neutrals:  (20549, 8)
Testing size:  (3888, 8)


(76653, 8)

In [16]:
labels=[]
for index, row in train_biden_df.iterrows():
    if row['avg_polarity'] == 0:
        labels.append(0)
    elif row['avg_polarity'] < 0:
        labels.append(-1)
    else:
        labels.append(1)

train_biden_df['label'] = labels
train_biden_df.head()

C:\Users\CK\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,user_id,tweet,state,likes,retweet_count,avg_polarity,hashtag,og_tweet,label
125784,3.494182e+09,thi #biden made #trumpisnotamerica http ubqafu,Montana,0.0,0.0,0.00000,biden,@IslandGirlPRV @BradBeauregardJ @MeidasTouch T...,0
125785,1.032807e+18,#censorship #hunterbiden #biden #bidenemail #b...,California,1.0,0.0,0.00000,biden,#censorship #HunterBiden #Biden #BidenEmails #...,0
125789,1.165118e+18,effort find truth about alleg that allow peopl...,Illinois,1.0,0.0,0.27115,biden,In an effort to find the truth about allegatio...,1
125790,1.139962e+18,twitter do everyth they help democrat elect ma...,California,1.0,0.0,0.12135,biden,Twitter is doing everything they can to help D...,1
125791,1.300837e+18,#bidencrimefamili #joebiden #hunterbiden #hunt...,California,0.0,0.0,0.00000,biden,@RealJamesWoods #BidenCrimeFamily #JoeBiden #H...,0


In [17]:
#combine labeled trump and biden datasets
train_df = pd.concat([train_trump_df, train_biden_df], ignore_index=True)
test_df = pd.concat([test_trump_df, test_biden_df], ignore_index=True)
display(train_df.head())
print(train_df.shape)
display(test_df.head())
print(test_df.shape)

,user_id,tweet,state,likes,retweet_count,avg_polarity,hashtag,og_tweet,label
0,8.436472e+06,#trump student use hear year year heard china ...,Oregon,2.0,1.0,0.461917,trump,"#Trump: As a student I used to hear for years,...",-1
1,4.741380e+07,#trump ralli #iowa http jjaluumh,District of Columbia,4.0,3.0,0.000000,trump,You get a tie! And you get a tie! #Trump ‘s ra...,0
2,1.138416e+09,minut were over long time omarosa never repres...,California,2.0,0.0,-0.323204,trump,@CLady62 Her 15 minutes were over long time ag...,1
3,5.404769e+08,singl most effect remedi erad anoth round #tru...,Pennsylvania,0.0,0.0,0.366371,trump,One of the single most effective remedies to e...,-1
4,1.243315e+18,#trump #presidenttrump #trump landslidevictori...,California,3.0,5.0,0.513350,trump,#Trump #PresidentTrump #Trump2020LandslideVict...,-1


(171263, 9)


,user_id,tweet,state,likes,retweet_count,avg_polarity,hashtag,og_tweet
0,9.007611e+17,there mani them unless have been vote more tha...,Ohio,0.0,0.0,-0.041564,trump,@DeeviousDenise @realDonaldTrump @nypost There...
1,8.436472e+06,#trump mind script right thi stuff want hear a...,Oregon,3.0,1.0,-0.003093,trump,"#Trump: You don't mind if I go off script, rig..."
2,1.087710e+09,well #kayleighmcenani taken twitter thi post r...,Georgia,3.0,4.0,-0.012900,trump,Well #kayleighmcenany was taken off of Twitter...
3,1.743698e+08,singl from demi lovato #commanderinchief caus ...,California,1.0,1.0,0.016378,trump,"The new single from Demi Lovato, #CommanderInC..."
4,8.436472e+06,thi time next week #trump go insist that didn ...,Oregon,1.0,1.0,0.012500,trump,By this time next week #Trump is going to be i...


(9586, 8)


In [18]:
def add_label(tweet):
    output = []
    for i, s in zip(tweet.index, tweet):
        output.append(LabeledSentence(s, ['tweet_', str(i)]))
    return output

In [19]:
train_labeled_tweets = add_label(train_df['tweet'])
test_labeled_tweets = add_label(test_df['tweet'])


C:\Users\CK\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  after removing the cwd from sys.path.


### Doc2Vec model creation

In [20]:
feature_size=200

In [21]:
d2v = gensim.models.Doc2Vec(dm=1, dm_mean=1, vector_size=feature_size, window=5, min_count=3, workers=32, alpha=0.1, seed=0)
d2v.build_vocab([i for i in tqdm(train_labeled_tweets)])
d2v.train(train_labeled_tweets, total_examples=len(train_df['tweet']), epochs=30)

100%|█████████████████████████████████████████████████████████████████████| 171263/171263 [00:00<00:00, 3982972.48it/s]


In [22]:
d2v_vectors = np.zeros((len(train_labeled_tweets), feature_size))
for i in range(len(train_df['tweet'])):
    d2v_vectors[i, :] = d2v.docvecs[i].reshape((1, feature_size))

d2v_df = pd.DataFrame(d2v_vectors)
print(d2v_df.shape)
d2v_df.head()

(171263, 200)


,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.110408,0.091677,0.041032,-0.085604,-0.090799,0.200027,0.096284,0.041122,0.145244,-0.026692,...,0.136696,0.231766,-0.119371,0.353117,-0.062682,-0.023983,-0.431572,0.228778,0.148602,-0.016786
1,-0.077525,-0.287868,0.316252,0.201448,-0.281335,-0.804113,0.273058,-0.317283,0.382776,-0.342576,...,-0.022986,0.064560,-0.112055,-0.364309,0.045115,0.213366,-0.864704,0.269889,0.152112,0.170484
2,0.302651,0.356875,-0.217220,-0.004243,0.238316,0.083669,-0.183905,-0.004662,-0.260197,-0.061375,...,0.002804,-0.108593,0.163901,0.158213,0.078153,-0.248236,-0.163710,0.003916,-0.112320,-0.094454
3,-0.237424,0.247700,0.222655,0.325715,-0.066750,-0.083057,-0.114325,-0.052503,-0.078669,0.005935,...,-0.110517,-0.031375,0.110939,-0.086917,-0.064546,0.023710,0.038219,-0.153389,0.081045,0.220044
4,-0.274879,-0.077842,-0.052642,0.057634,-0.197947,0.065058,-0.106758,-0.288664,0.080520,0.002223,...,0.013840,-0.070174,-0.032054,-0.078490,0.091314,0.036802,-0.061398,-0.101887,-0.135337,-0.268130


### Word2Vec model creation

In [23]:
split_tokens = train_df['tweet'].apply(lambda x: x.split())
w2v = gensim.models.Word2Vec(split_tokens, size=feature_size, window=5, min_count=3, sg=1, workers=32, seed=0)
w2v.train(split_tokens, total_examples=len(train_df['tweet']), epochs=30)

(62336714, 79142010)

##### Represent tweet using average of vectors of words in the tweet

In [24]:
def avg_vector(w2v, tweet_tokens, size):
    vec_sum = np.zeros(size).reshape((1, size))
    num_tokens=0
    for token in tweet_tokens:
        try:
            vec_sum += w2v[token].reshape((1, size))
            num_tokens+=1;
        except KeyError:
            continue
    if num_tokens>0:
        vec_sum = vec_sum/num_tokens
    
    return vec_sum

In [25]:
w2v_vectors = np.zeros((len(split_tokens), feature_size))
for i in range(len(split_tokens)):
    w2v_vectors[i, :]=avg_vector(w2v, split_tokens[i], feature_size)
w2v_df = pd.DataFrame(w2v_vectors)
print(w2v_df.shape)
w2v_df.head()

C:\Users\CK\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


(171263, 200)


,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,-0.019483,-0.116082,-0.041735,0.032281,-0.037267,-0.065538,0.084294,0.062075,-0.084221,0.015691,...,0.067189,-0.021923,-0.024689,0.081614,-0.034112,0.142319,0.214829,0.065318,0.110041,-0.023829
1,0.139823,-0.023224,-0.182978,-0.107680,0.214705,0.019800,0.015048,-0.166629,-0.057950,-0.246714,...,0.178744,-0.092041,-0.246127,0.197027,0.129968,0.021253,0.459367,-0.019741,-0.090035,-0.015550
2,-0.038944,-0.131563,0.065573,0.072883,0.081996,-0.127323,0.035462,0.026877,-0.114838,0.116959,...,-0.025540,0.009507,-0.019766,0.049861,-0.103476,0.001360,0.284049,0.046093,0.092531,-0.040536
3,-0.088081,-0.066113,0.009789,0.108681,0.108755,-0.071762,-0.037451,0.036803,-0.020722,0.063148,...,-0.047877,-0.204524,-0.013202,0.117880,-0.224815,0.104097,0.298107,-0.021776,-0.069497,-0.078719
4,-0.028159,-0.101266,-0.129553,-0.039737,0.058104,-0.070536,0.085577,0.035558,-0.236812,-0.019030,...,-0.043926,-0.475411,-0.459329,0.098810,0.153287,-0.176517,0.275078,0.116078,-0.137998,-0.429426


In [26]:
#splitting xtrain and ytrain, xvaild, yvalid
w2v_xtrain = w2v_df.iloc[:140000, :]
w2v_xvalid = w2v_df.iloc[140000:, :]

# d2v_xtrain = d2v_df.iloc[:140000, :]
# d2v_xvalid = d2v_df.iloc[140000:, :]

yvalid = train_df.iloc[140000:, :]['label']
ytrain = train_df.iloc[:140000, :]['label']
set(ytrain)

{-1, 0, 1}

In [27]:
d2v_full_df = d2v_df
d2v_full_df['label'] = train_df['label']
d2v_full_df.to_csv('d2v_df.csv', index=False)

In [28]:
w2v_full_df = w2v_df
w2v_full_df['label'] = train_df['label']
w2v_full_df.to_csv('w2v_df.csv', index=False)

### SVM attempt

In [29]:
# svc = svm.SVC(kernel='poly', C=1, probability=True).fit(w2v_xtrain, ytrain) 
# prediction = svc.predict_proba(w2v_xvalid) 
# prediction_int = prediction[:,1] >= 0.3 
# prediction_int = prediction_int.astype(np.int) 
# f1_score(yvalid, prediction_int)

In [30]:
# svc = svm.SVC(kernel='poly', C=1, probability=True).fit(d2v_xtrain, ytrain) 
# prediction = svc.predict_proba(d2v_xvalid) 
# prediction_int = prediction[:,1] >= 0.3 
# prediction_int = prediction_int.astype(np.int) 
# f1_score(yvalid, prediction_int)

### Random Tree Attempt

In [31]:
# w2v_rf = RandomForestClassifier(n_estimators=500, random_state=0).fit(w2v_xtrain, ytrain)
# w2v_prediction = w2v_rf.predict(w2v_xvalid)
# f1_score(yvalid, w2v_prediction)

In [32]:
# d2v_rf = RandomForestClassifier(n_estimators=500, random_state=11).fit(w2v_xtrain, ytrain)
# d2v_prediction = d2v_rf.predict(d2v_xvalid)
# f1_score(yvalid, d2v_prediction)

### XGBoost attempt

In [33]:
!pip install xgboost
from xgboost import XGBClassifier

In [34]:
w2v_xgb = XGBClassifier(max_depth=5, n_estimators=1000, nthread=3).fit(w2v_xtrain, ytrain)
w2v_prediction = w2v_xgb.predict(w2v_xvalid)
f1_score(yvalid, w2v_prediction, average='weighted')

0.5723147118676102

In [38]:
d2v_xgb = XGBClassifier(max_depth=5, n_estimators=1000, nthread=3).fit(d2v_xtrain, ytrain)
d2v_prediction = d2v_xgb.predict(d2v_xvalid)
f1_score(yvalid, d2v_prediction, average='weighted')

0.3344405546075232

In [ ]:
# no need because d2v f1score is bad
#d2v = gensim.models.Doc2Vec(dm=1, dm_mean=1, vector_size=feature_size, window=5, min_count=3, workers=32, alpha=0.1, seed=0)
# d2v.build_vocab([i for i in tqdm(test_labeled_tweets)])
# d2v.train(test_labeled_tweets, total_examples=len(train_df['tweet']), epochs=30)

# d2v_vectors = np.zeros((len(test_labeled_tweets), feature_size))
# for i in range(len(test_df['tweet'])):
#     d2v_vectors[i, :] = d2v.docvecs[i].reshape((1, feature_size))

# test_d2v_df = pd.DataFrame(d2v_vectors)
# print(test_d2v_df.shape)
# test_d2v_df.head()

In [35]:
split_tokens = test_df['tweet'].apply(lambda x: x.split())
w2v = gensim.models.Word2Vec(split_tokens, size=feature_size, window=5, min_count=3, sg=1, workers=32, seed=0)
w2v.train(split_tokens, total_examples=len(test_df['tweet']), epochs=30)

w2v_vectors = np.zeros((len(split_tokens), feature_size))
for i in range(len(split_tokens)):
    w2v_vectors[i, :]=avg_vector(w2v, split_tokens[i], feature_size)
test_w2v_df = pd.DataFrame(w2v_vectors)
print(test_w2v_df.shape)
test_w2v_df.head()

C:\Users\CK\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


(9586, 200)


,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,-0.118253,-0.077265,-0.309727,0.080739,0.036182,-0.181144,0.211139,-0.225574,-0.098953,-0.018838,...,0.082228,-0.171528,-0.159183,0.320139,-0.053437,0.232577,0.424878,0.015796,-0.024923,-0.329314
1,-0.074638,-0.171072,-0.236927,-0.094039,0.063383,-0.169976,-0.090608,-0.001959,-0.125327,-0.083240,...,-0.015820,-0.148579,-0.091527,0.248047,0.059412,0.147735,0.280832,-0.006078,-0.034239,0.007589
2,-0.242561,-0.002576,-0.114294,-0.224185,0.109051,-0.173046,0.091103,-0.059433,-0.022843,-0.035010,...,0.181333,-0.439795,-0.131820,0.213074,0.187139,0.223949,0.356128,0.262993,-0.113905,-0.228715
3,-0.070555,-0.087311,-0.255469,-0.088785,-0.012836,-0.140091,0.128457,0.116059,-0.156266,-0.084481,...,0.057451,-0.096182,-0.192537,0.352246,-0.039264,0.280799,0.386077,0.076305,-0.003611,-0.156745
4,-0.043660,0.053978,-0.251400,0.086851,-0.039278,-0.060640,0.274531,-0.127011,0.074176,-0.099337,...,0.176237,-0.125376,-0.130484,0.271970,-0.053617,0.189223,0.219852,0.125715,-0.016714,-0.178117


In [36]:
import xgboost as xgb

In [37]:
ytrain=ytrain +1
yvalid=yvalid +1
dtrain = xgb.DMatrix(w2v_xtrain, label=ytrain)
dvalid = xgb.DMatrix(w2v_xvalid, label=yvalid)
dtest = xgb.DMatrix(test_w2v_df)
params={
    'objective': 'multi:softprob',
    'max_depth': '5',
    'min_child_weight': 1,
    'eta': 0.3,
    'subsample': 1,
    'colsample': 1,
    'num_class': 3
}


In [38]:
def custom_eval(preds, dtrain):
    labels = dtrain.get_label().astype(np.int)
    preds = np.argmax(preds, axis=1)
    return [('f1_score', f1_score(labels, preds, average='weighted'))]

gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(6,8)
    for min_child_weight in range(5,8)
    ]

max_f1 = 0. 

best_params = None 

for max_depth, min_child_weight in gridsearch_params:    
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    cv_results = xgb.cv(
        params,
        dtrain,
        feval= custom_eval,
        num_boost_round=200,
        seed=16,
        nfold=5,
        )     
       
mean_f1 = cv_results['test-f1_score-mean'].max()
boost_rounds = cv_results['test-f1_score-mean'].idxmax()     

if mean_f1 > max_f1:
        max_f1 = mean_f1
        best_params = (max_depth,min_child_weight) 
        
print("Best max depth: ", best_params[0])
print("Best min_child_weight: ", best_params[1])
print("f1_score", max_f1)


Best max depth:  7
Best min_child_weight:  6
f1_score 0.6538558


In [39]:
gridsearch_params = [
    (subsample)
    for subsample in [i/10. for i in range(8,11)]
]

max_f1 = 0. 
best_params = None 

for subsample in gridsearch_params:
    params['subsample'] = subsample
    
    cv_results = xgb.cv(
        params,
        dtrain,
        feval= custom_eval,
        num_boost_round=200,
        maximize=True,
        seed=16,
        nfold=5,
        early_stopping_rounds=10
        )
    mean_f1 = cv_results['test-f1_score-mean'].max()
    boost_rounds = cv_results['test-f1_score-mean'].idxmax()
    
    if mean_f1 > max_f1:
        max_f1 = mean_f1
        best_params = (subsample) 

print("best subsample ", best_params)

best subsample  1


In [43]:
best_params = None 
for eta in [.3, .2, .1, .05]:
    params['eta'] = eta
    cv_results = xgb.cv(
        params,
        dtrain,
        feval= custom_eval,
        num_boost_round=1000,
        maximize=True,
        seed=16,
        nfold=5,
        early_stopping_rounds=10
    )

    mean_f1 = cv_results['test-f1_score-mean'].max()
    boost_rounds = cv_results['test-f1_score-mean'].idxmax()    
    if mean_f1 > max_f1:
        max_f1 = mean_f1
        best_params = eta 
        
print("Best eta: ", best_params)

Best eta:  0.3


In [44]:
params={
    'objective': 'multi:softprob',
    'max_depth': '7',
    'min_child_weight': 6,
    'eta': 0.3,
    'subsample': 1,
    'num_class': 3
}

In [45]:

xgb_w2v = xgb.train(params, dtrain, num_boost_round=1000,  maximize=True, evals=[(dvalid, "Validation")], early_stopping_rounds=10)

[0]	Validation-merror:0.50379
Will train until Validation-merror hasn't improved in 10 rounds.
[1]	Validation-merror:0.48514
[2]	Validation-merror:0.48920
[3]	Validation-merror:0.48927
[4]	Validation-merror:0.48933
[5]	Validation-merror:0.49061
[6]	Validation-merror:0.49691
[7]	Validation-merror:0.49816
[8]	Validation-merror:0.49432
[9]	Validation-merror:0.49397
[10]	Validation-merror:0.49432
Stopping. Best iteration:
[0]	Validation-merror:0.50379



In [46]:
test_pred = xgb_w2v.predict(dtest)
test_label = []
for pred in test_pred:
    test_label.append(np.argmax(pred) - 1)
len(test_label)


9586

In [47]:
test_df['label'] = test_label
test_df.head()


,user_id,tweet,state,likes,retweet_count,avg_polarity,hashtag,og_tweet,label
0,9.007611e+17,there mani them unless have been vote more tha...,Ohio,0.0,0.0,-0.041564,trump,@DeeviousDenise @realDonaldTrump @nypost There...,1
1,8.436472e+06,#trump mind script right thi stuff want hear a...,Oregon,3.0,1.0,-0.003093,trump,"#Trump: You don't mind if I go off script, rig...",1
2,1.087710e+09,well #kayleighmcenani taken twitter thi post r...,Georgia,3.0,4.0,-0.012900,trump,Well #kayleighmcenany was taken off of Twitter...,1
3,1.743698e+08,singl from demi lovato #commanderinchief caus ...,California,1.0,1.0,0.016378,trump,"The new single from Demi Lovato, #CommanderInC...",1
4,8.436472e+06,thi time next week #trump go insist that didn ...,Oregon,1.0,1.0,0.012500,trump,By this time next week #Trump is going to be i...,-1


In [48]:
fin_full_df = pd.concat([train_df, test_df], ignore_index=True)
fin_full_df.shape
fin_full_df.head()

,user_id,tweet,state,likes,retweet_count,avg_polarity,hashtag,og_tweet,label
0,8.436472e+06,#trump student use hear year year heard china ...,Oregon,2.0,1.0,0.461917,trump,"#Trump: As a student I used to hear for years,...",-1
1,4.741380e+07,#trump ralli #iowa http jjaluumh,District of Columbia,4.0,3.0,0.000000,trump,You get a tie! And you get a tie! #Trump ‘s ra...,0
2,1.138416e+09,minut were over long time omarosa never repres...,California,2.0,0.0,-0.323204,trump,@CLady62 Her 15 minutes were over long time ag...,1
3,5.404769e+08,singl most effect remedi erad anoth round #tru...,Pennsylvania,0.0,0.0,0.366371,trump,One of the single most effective remedies to e...,-1
4,1.243315e+18,#trump #presidenttrump #trump landslidevictori...,California,3.0,5.0,0.513350,trump,#Trump #PresidentTrump #Trump2020LandslideVict...,-1


In [49]:
final_df = fin_full_df[['user_id','og_tweet', 'state', 'likes', 'retweet_count', 'avg_polarity', 'label', 'hashtag']]
final_df.columns = ['user_id', 'tweet', 'state', 'likes', 'retweet_count', 'avg_polarity', 'label', 'hashtag']
print(final_df.shape)
print(full_df.shape)
final_df.tail(20)

(180849, 8)
(180849, 7)


,user_id,tweet,state,likes,retweet_count,avg_polarity,label,hashtag
180829,1.244982e+18,America will not forget how the Republicans st...,Ohio,3.0,1.0,-0.015250,0,biden
180830,1.244982e+18,America will not forget how the Republicans st...,Ohio,2.0,1.0,-0.015250,0,biden
180831,1.244982e+18,America will not forget how the Republicans st...,Ohio,2.0,2.0,-0.015250,0,biden
180832,1.244982e+18,America will not forget how the Republicans st...,Ohio,2.0,2.0,-0.015250,0,biden
180833,1.244982e+18,America will not forget how the Republicans st...,Ohio,12.0,9.0,-0.015250,0,biden
180834,1.244982e+18,America will not forget how the Republicans st...,Ohio,1.0,1.0,-0.015250,0,biden
180835,1.244982e+18,America will not forget how the Republicans st...,Ohio,1.0,1.0,-0.015250,0,biden
180836,1.244982e+18,America will not forget how the Republicans st...,Ohio,1.0,1.0,-0.015250,0,biden
180837,1.244982e+18,America will not forget how the Republicans st...,Ohio,10.0,8.0,-0.015250,0,biden
180838,3.327421e+09,194 Reasons ...\n#GayFolk #GayVeterans #VetsRe...,Washington,4.0,1.0,-0.050000,1,biden


In [50]:
fin_full_df.shape
fin_full_df = fin_full_df.drop(columns=['tweet', 'hashtag'])

In [51]:
fin_full_df['tweet'] = og_tweet
full_df['label']=fin_full_df['label']

In [52]:
final_df.to_csv('combined_labeled_df.csv', index=False)